In [1]:
import geopandas as gpd

In [13]:
stations_context = gpd.read_file("../subway_stations_500_buffer.geojson")
stations_gaps = gpd.read_file("./subway_station_ada_gaps.csv")
stations_centrality = gpd.read_file("./subway_station_centrality.csv")

In [26]:
stations_base = stations_context.reindex(
    [
        "name",
        "line",
        "complex_id",
        "gtfs_stop_id",
        "ada_status_code",
        "ridership_2019",
        "geometry",
    ],
    axis=1,
)
stations_ada_neighbor_gap = stations_gaps.reindex(
    ["complex_id", "nearest_accessible_neighbor"], axis=1
)
stations_betweenness_centrality = stations_centrality.reindex(
    ["complex_id", "centrality"], axis=1
)

In [42]:
stations = stations_base.join(
    stations_ada_neighbor_gap.set_index("complex_id"), on="complex_id"
)
stations = stations.join(
    stations_betweenness_centrality.set_index("complex_id"), on="complex_id"
)
stations = stations.rename(
    columns={
        "line": "lines",
        "nearest_accessible_neighbor": "ada_neighbor_gap",
        "centrality": "betweenness_centrality",
        "ridership_2019": "ridership",
    }
)

In [44]:
stations.to_file("subway_stations.geojson", driver="GeoJSON")